In [1]:
import os
os.getcwd()

'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project/research'

In [2]:
os.chdir("../.")
os.getcwd()

'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationconfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [4]:
from textsummarizer.constant import *
from textsummarizer.utils.common import read_yaml,create_directories

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationconfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config =  DataValidationconfig(
                root_dir=config.root_dir,
                source_url = config.source_url,
                local_data_file = config.local_data_file,
                unzip_dir =config.unzip_dir)
        return data_ingestion_config


In [6]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self,config:DataIngetionconfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url = self.config.source_url,filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded! with following headers: {headers}")
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_file))}")


    
    def unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"unzip successfully")

In [19]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    logger.error(e)

[2024-01-20 21:44:27,688: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-01-20 21:44:27,694: INFO: common: yaml file:params.yaml loaded successfully]
[2024-01-20 21:44:27,695: INFO: common: directory:artifacts already exists]
[2024-01-20 21:44:27,695: INFO: common: directory:artifacts/data_ingestion already exists]
[2024-01-20 21:44:27,698: INFO: 146028700: file already exists of size 7718 KB]
[2024-01-20 21:44:27,831: INFO: 146028700: unzip successfully]
